# Inventory utils

> Some additional classes and functions that help building inventory management environments.

In [ ]:
#| default_exp envs.inventory.inventory_utils

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export

import logging


from abc import ABC, abstractmethod
from typing import Union, Tuple, List, Literal

from ddopai.envs.base import BaseEnvironment
from ddopai.utils import Parameter, MDPInfo
from ddopai.dataloaders.base import BaseDataLoader
from ddopai.loss_functions import pinball_loss
from ddopai.utils import set_param, Parameter


import gymnasium as gym

import numpy as np
import time

In [ ]:
#| export
class OrderPipeline():
   
    """
    Class to handle the order pipeline in the inventory environments. It is used to keep track of the orders
    that are placed. It can account for fixed and variable lead times.
    
    """

    def __init__(self, 

        num_units: int,  # number of units (SKUs)
        lead_time_mean: Parameter | np.ndarray | List | int | float,  # mean lead time
        lead_time_stochasticity: Literal["fixed", "gamma", "normal_absolute", "normal_relative"] = "fixed", # "fixed", "gamma", "normal_absolute", "normal_relative"
        lead_time_variance: Parameter | np.ndarray | List | int | float | None = None,  # variance of the lead time
        max_lead_time: list[object] | None = None,  # maximum lead time in case of stochastic lead times
        min_lead_time: list[object] | None = 1,  # minimum lead time in case of stochastic lead times

        ) -> None:

        self.set_param('num_units', num_units, shape=(1,), new=True)
        self.set_param('lead_time_mean', lead_time_mean, shape=(self.num_units[0],), new=True)
        self.set_param('lead_time_variance', lead_time_variance, shape=(self.num_units[0],), new=True)
        self.lead_time_stochasticity = lead_time_stochasticity
        self.check_stochasticity(max_lead_time)

        if max_lead_time is None:
            self.set_param('max_lead_time', lead_time_mean, shape=(self.num_units[0],), new=True)
        else:
            self.set_param('max_lead_time', max_lead_time, shape=(self.num_units[0],), new=True)
        self.set_param('min_lead_time', min_lead_time, shape=(self.num_units[0],), new=True)

        if self.max_lead_time is None:
            self.max_lead_time = self.lead_time_mean
        if self.min_lead_time is None:
            self.min_lead_time = 1

        self.check_max_min_mean_lt()
  
        self.pipeline = np.zeros((np.max(self.max_lead_time), num_units))
        self.lead_time_realized = np.zeros((np.max(self.max_lead_time), num_units))

    def get_pipeline(self) -> np.ndarray:
        """ Get the current pipeline """

        return self.pipeline

    def reset(self) -> None:
        """ Reset the pipeline """

        self.pipeline = np.zeros((np.max(self.max_lead_time), self.num_units[0]))
        self.lead_time_realized = np.zeros((np.max(self.max_lead_time), self.num_units[0]))


    def step(self, 
        orders: np.ndarray,
        ) -> np.ndarray:
        
        """ Add orders to the pipeline and return the orders that are arriving """


        # print("beginnig pipeline:")
        # print(self.pipeline)
        # print("beginnig lead_time_realized:")
        # print(self.lead_time_realized)


        orders_arriving = self.get_orders_arriving().copy()
        lead_times = self.draw_lead_times().copy()

        # print("orders_arriving:")
        # print(orders_arriving)
        # print("new orders:")
        # print(orders)
        # print("new lead_times:")
        # print(lead_times)


        self.pipeline = np.roll(self.pipeline, -1, axis=0)
        self.lead_time_realized = np.roll(self.lead_time_realized, -1, axis=0)
        self.pipeline[-1, :] = 0
        self.lead_time_realized[-1, :] = 0
        
        self.pipeline[-1, :] = orders.copy()
        self.lead_time_realized[-1, :] = lead_times
        self.lead_time_realized -= 1
        self.lead_time_realized = np.clip(self.lead_time_realized, 0, None)

        # print("ending pipeline:")
        # print(self.pipeline)
        # print("ending lead_time_realized:")
        # print(self.lead_time_realized)

        return orders_arriving

    def get_orders_arriving(self) -> np.ndarray:

        """ Get the orders that are arriving in the current period """

        orders_arriving = np.zeros(self.num_units[0])
        for i in range(self.num_units[0]):
            # check along the pipeline where the lead time is 0

            arriving_indices = np.where(self.lead_time_realized[:, i] == 0)[0]

            orders_arriving[i] = np.sum(self.pipeline[arriving_indices, i])

            self.pipeline[arriving_indices, i] = 0

        return orders_arriving

    def draw_lead_times(self) -> np.ndarray:
        """ Draw lead times for the orders """

        if self.lead_time_stochasticity == "fixed":
            lead_times = self.lead_time_mean
        elif self.lead_time_stochasticity == "gamma":
            lead_times = np.random.gamma(self.lead_time_mean, 1, self.num_units[0])
        elif self.lead_time_stochasticity == "normal_absolute":
            lead_times = np.random.normal(self.lead_time_mean, self.lead_time_variance, self.num_units[0])
        elif self.lead_time_stochasticity == "normal_relative":
            lead_times = np.random.normal(self.lead_time_mean, self.lead_time_mean * self.lead_time_variance, self.num_units[0])
        else:
            raise ValueError("Invalid lead time stochasticity")

        lead_times = np.clip(lead_times, self.min_lead_time, self.max_lead_time)
        lead_times = np.round(lead_times).astype(int)

        return lead_times

    def check_stochasticity(self, max_lead_time):
        """ Check that params for stochastic lead times are set correctly """

        # lead time mean to be set in any case (it will be the determinstic lead time if lead_time_stochasticity is fixed)
        if self.lead_time_mean is None:
            raise ValueError("Lead time mean is not set")
        if self.lead_time_stochasticity == "fixed" or self.lead_time_stochasticity == "gamma":
            if self.lead_time_variance is not None:
                raise ValueError("Lead time variance must be None for fixed lead times (no variance) or gamma lead times (variance is set by the gamma distribution)")
        elif self.lead_time_stochasticity == "normal_absolute" or self.lead_time_stochasticity == "normal_relative":
            if self.lead_time_variance is None:
                raise ValueError("Lead time variance must be set for normal lead times")
        else:
            raise ValueError("Invalid lead time stochasticity")
        
        if self.lead_time_stochasticity != "fixed":
            if max_lead_time is None:
                raise ValueError("Max lead time must be set for stochastic lead times")

    def check_max_min_mean_lt(self):
        if np.any(self.max_lead_time < self.lead_time_mean):
            raise ValueError("Max lead time must be greater than or equal to the lead time mean")
        if np.any(self.min_lead_time < 1):
            raise ValueError("Min lead time must be at least 1")
        if np.any(self.min_lead_time > self.lead_time_mean):
            raise ValueError("Min lead time must be less than or equal to the lead time mean")
        if np.any(self.max_lead_time < self.min_lead_time):
            raise ValueError("Max lead time must be greater than or equal to the min lead time")
        if np.any(self.max_lead_time < 1):
            raise ValueError("Max lead time must be at least 1")
        if np.any(self.lead_time_mean < 1):
            raise ValueError("Lead time mean must be at least 1")
            
        
    def set_param(self,
                        name: str, # name of the parameter (will become the attribute name)
                        input: Parameter | int | float | np.ndarray | List, # input value of the parameter
                        shape: tuple = (1,), # shape of the parameter
                        new: bool = False # whether to create a new parameter or update an existing one
                        ) -> None: #
        
        """
        Set a parameter for the environment. It converts scalar values to numpy arrays and ensures that
        environment parameters are either of the Parameter class of Numpy arrays. If new is set to True, 
        the function will create a new parameter or update an existing one otherwise. If new is set to
        False, the function will raise an error if the parameter does not exist.
        """

        set_param(self, name, input, shape, new)

    @property
    def shape(self) -> Tuple:
        """ Get the shape of the pipeline """

        return self.pipeline.shape


In [ ]:
#| hide
# testpipeline = OrderPipeline(
#                             num_units=2,
#                             lead_time_mean=[2, 3],
#                             max_lead_time=[5, 5],
#                             lead_time_variance=[.25],
#                             lead_time_stochasticity="normal_relative",

# )

# testpipeline.pipeline

# orders_arriving = testpipeline.step(np.array([1, 2]))
# orders_arriving = testpipeline.step(np.array([3, 4]))
# orders_arriving = testpipeline.step(np.array([5, 6]))
# orders_arriving = testpipeline.step(np.array([7, 8]))
# orders_arriving = testpipeline.step(np.array([9, 10]))

In [ ]:
show_doc(OrderPipeline, title_level=2)

---

## OrderPipeline

>      OrderPipeline (num_units:int, lead_time_mean:Union[ddopai.utils.Parameter
>                     ,numpy.ndarray,List,int,float], lead_time_stochasticity:Li
>                     teral['fixed','gamma','normal_absolute','normal_relative']
>                     ='fixed', lead_time_variance:Union[ddopai.utils.Parameter,
>                     int,float,numpy.ndarray,List,NoneType]=None,
>                     max_lead_time:list[object]|None=None,
>                     min_lead_time:list[object]|None=1)

*Class to handle the order pipeline in the inventory environments. It is used to keep track of the orders
that are placed. It can account for fixed and variable lead times.*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| num_units | int |  | number of units (SKUs) |
| lead_time_mean | Union |  | mean lead time |
| lead_time_stochasticity | Literal | fixed | "fixed", "gamma", "normal_absolute", "normal_relative" |
| lead_time_variance | Union | None | variance of the lead time |
| max_lead_time | list[object] \| None | None | maximum lead time in case of stochastic lead times |
| min_lead_time | list[object] \| None | 1 | minimum lead time in case of stochastic lead times |
| **Returns** | **None** |  |  |

In [ ]:
show_doc(OrderPipeline.get_pipeline, title_level=3)

---

### OrderPipeline.get_pipeline

>      OrderPipeline.get_pipeline ()

*Get the current pipeline*

In [ ]:
show_doc(OrderPipeline.reset, title_level=3)

---

### OrderPipeline.reset

>      OrderPipeline.reset ()

*Reset the pipeline*

In [ ]:
show_doc(OrderPipeline.step, title_level=3)

---

### OrderPipeline.step

>      OrderPipeline.step (orders:numpy.ndarray)

*Add orders to the pipeline and return the orders that are arriving*

In [ ]:
show_doc(OrderPipeline.get_orders_arriving, title_level=3)

---

### OrderPipeline.get_orders_arriving

>      OrderPipeline.get_orders_arriving ()

*Get the orders that are arriving in the current period*

In [ ]:
show_doc(OrderPipeline.draw_lead_times, title_level=3)

---

### OrderPipeline.draw_lead_times

>      OrderPipeline.draw_lead_times ()

*Draw lead times for the orders*

In [ ]:
show_doc(OrderPipeline.check_stochasticity, title_level=3)

---

### OrderPipeline.check_stochasticity

>      OrderPipeline.check_stochasticity (max_lead_time)

*Check that params for stochastic lead times are set correctly*

In [ ]:
show_doc(OrderPipeline.check_max_min_mean_lt, title_level=3)

---

### OrderPipeline.check_max_min_mean_lt

>      OrderPipeline.check_max_min_mean_lt ()

In [ ]:
show_doc(OrderPipeline.set_param, title_level=3)

---

### OrderPipeline.set_param

>      OrderPipeline.set_param (name:str,
>                               input:Union[ddopai.utils.Parameter,int,float,num
>                               py.ndarray,List], shape:tuple=(1,),
>                               new:bool=False)

*Set a parameter for the environment. It converts scalar values to numpy arrays and ensures that
environment parameters are either of the Parameter class of Numpy arrays. If new is set to True, 
the function will create a new parameter or update an existing one otherwise. If new is set to
False, the function will raise an error if the parameter does not exist.*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| name | str |  | name of the parameter (will become the attribute name) |
| input | Union |  | input value of the parameter |
| shape | tuple | (1,) | shape of the parameter |
| new | bool | False | whether to create a new parameter or update an existing one |
| **Returns** | **None** |  |  |

In [ ]:
show_doc(OrderPipeline.shape, title_level=3)

---

### OrderPipeline.shape

>      OrderPipeline.shape ()

*Get the shape of the pipeline*

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()